<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/50_4_Speech_synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка необходимых библиотек
%%capture
!pip install transformers gtts ipython

In [2]:
# Импорт библиотек
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from gtts import gTTS
from IPython.display import Audio, display
import os

In [ ]:
# Список команд для распознавания
COMMANDS = [
    "hello", "what's the time", "tell me a joke", "how are you",
    "what's your name", "goodbye", "thank you", "play music",
    "stop", "help"]

# Инициализация языковой модели для генерации ответов
model_name = "facebook/blenderbot_small-90M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Функция для генерации ответа на команду
def generate_response(command):
    # Создаем контекст для модели
    if command.lower() in COMMANDS:
        # Генерируем ответ в зависимости от команды
        if command.lower() == "hello":
            return "Hello there! How can I help you today?"
        elif command.lower() == "what's the time":
            return "I'm sorry, I don't have access to the current time in this environment."
        elif command.lower() == "tell me a joke":
            return "Why don't scientists trust atoms? Because they make up everything!"
        elif command.lower() == "how are you":
            return "I'm just a computer program, but I'm functioning well, thank you for asking!"
        elif command.lower() == "what's your name":
            return "I'm an AI assistant created to help you with your tasks."
        elif command.lower() == "goodbye":
            return "Goodbye! Have a great day!"
        elif command.lower() == "thank you":
            return "You're welcome! Is there anything else I can help you with?"
        elif command.lower() == "play music":
            return "I'm sorry, I can't play music directly, but I can recommend some!"
        elif command.lower() == "stop":
            return "Okay, stopping now."
        elif command.lower() == "help":
            return "I can respond to these commands: " + ", ".join(COMMANDS)
    else:
        # Если команда не распознана, используем языковую модель для генерации ответа
        inputs = tokenizer([command], return_tensors="pt", padding=True)
        reply_ids = model.generate(**inputs)
        response = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
        return response

# Функция для преобразования текста в речь и воспроизведения
def text_to_speech(text, lang='en'):

    # Создаем временный файл
    tts = gTTS(text=text, lang=lang, slow=False)
    filename = "response.mp3"
    tts.save(filename)

    # Воспроизводим аудио
    display(Audio(filename, autoplay=True))

    # Возвращаем имя файла на случай, если нужно его сохранить
    return filename

# Основной цикл взаимодействия
def main():
    print("Available commands:", ", ".join(COMMANDS))
    print("Type 'exit' to quit the program.")

    while True:
        # Получаем ввод от пользователя
        user_input = input("You: ").strip()

        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        # Генерируем ответ
        response = generate_response(user_input)
        print("AI:", response)

        # Преобразуем ответ в речь и воспроизводим
        audio_file = text_to_speech(response)

        # Удаляем временный файл после использования
        if os.path.exists(audio_file):
            os.remove(audio_file)

# Запускаем программу
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/964k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/345k [00:00<?, ?B/s]

...

Available commands: hello, what's the time, tell me a joke, how are you, what's your name, goodbye, thank you, play music, stop, help
Type 'exit' to quit the program.


model.safetensors:   0%|          | 0.00/350M [00:00<?, ?B/s]

You: hello
AI: Hello there! How can I help you today?


You: stop
AI: Okay, stopping now.


You: we can go to the park today together?
AI: we can go to the park today together ? regulation regulation regulation burregulation regulation muregulation regulation prregulation regulation gm regulation gm gm gm odgm gm draft gm gm aki gm aki aki aki gm draft aki aki ododgm aki odgm draft draft draft aki oddraft draft oddraft aki draft draft gm draft odododdraft gm oddraft drafted draft draft rowdraft draft qdraft draft drafted oddraft qododqoddraft odqdraft odordraft draft mudraft draft burdraft draft ord draft draft ordraft oddrafted draft odmudraft odburdraft odprdraft draft potter draft draft prdraft qqqdraft q@@
